In [ ]:
import os
import cv2
import math
import torch
import shutil 
import random
import numpy as np 
import pandas as pd 
from tqdm import tqdm
from PIL import Image
import torch.nn as nn
from torch import cuda
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from skimage.color import rgb2lab, lab2rgb
import torchvision.models as models

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
import shutil
def save_model(model):
  drive.mount('/content/drive')
  shutil.copy("/content/" + model, "/content/drive/MyDrive/cv thesis/Automatic-Temporal-Video-Colorization-Net(ATVC-Net)/trained_models")
  print("Model Saved")
  drive.flush_and_unmount()

def load_model(model):
  drive.mount('/content/drive')
  shutil.copy("/content/drive/MyDrive/cv thesis/Automatic-Temporal-Video-Colorization-Net(ATVC-Net)/trained_models/" + model, '/content/')
  print("Model Loaded")
  drive.flush_and_unmount()

In [ ]:
!pip install --upgrade --no-cache-dir gdown
# !gdown --id 1rE8R5lW1C-0nBCdJ-48DpEKJREXx9uOa #Davis Datset
!gdown --id 1V68xX9CyfPPhgqAzwZTmnfaJhXK7DMdd #network.py
!gdown --id 1Mgvfb9BfIjmRH3G9ju0NHWhKOpvbF5mS #loss.py
!gdown --id 1Ta1BoQRk9GP0BoME86wro8kk3o6f86rb #dataset.py

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1V68xX9CyfPPhgqAzwZTmnfaJhXK7DMdd
To: /content/network.py
100% 4.86k/4.86k [00:00<00:00, 8.87MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Mgvfb9BfIjmRH3G9ju0NHWhKOpvbF5mS
To: /content/loss.py
100% 4.80k/4.80k [00:00<00:00, 9.08MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://

In [ ]:
# !gdown --id 1-k5sNFrq7K1ZSfbxIHGDTtSLtgCxY310 #Generator finetuned epoch 10
# !gdown --id 10-CNpZ5L4LzSd4OHlfxWiqtzHXXkq-ZQ #Discriminator trained epoch 20

# !gdown --id 1-Ox_MPhTSrk183jSCP-xVDPpaFD9-1He #Discriminator trained epoch 10
!gdown --id 1xJ0zPyBr3c6rnG4HQfMk7B64y61YLQEv #resnet - gray
!gdown --id 1-jFyCtViLuwmRlfepHKKIdgFjvDvPlTS #netG original pretrained
# !gdown --id 1Csq7o2JqQjUkmPM4JPFwPIA74-_ik5Mt #sampled image_net
# !gdown --id 1-Cl9_LZcZqdGTVY2CVlAk0ksPIlL0Fsd #net D 5 epoch
# !gdown --id 1--YexYubcUKt-HiHBIhI6lxtzAABVDNE #net G 5 epoch

In [ ]:
!gdown --id 1TOCFl-LcCs9QhZT0KZDtpddpii1Si-x8 #SElf made dataset 8.2k
!gdown --id 13O5OzhXQaU2KZvIaC9HHHYiGbMdxHCOZ #annotations

In [ ]:
load_model("netG_90.pt")
load_model("netD_90.pt")

Mounted at /content/drive
Model Loaded
Mounted at /content/drive
Model Loaded


In [ ]:
!ls


checkpoint-trained.pth.tar   loss.py		   network.py
colorization_annotation.csv  __MACOSX		   __pycache__
Colorization_Data.zip	     madhubala_frames	   resnet_gray_weights.pth.tar
dataset.py		     madhubala_frames.zip  sample_data
foo.png			     netD_90.pt
Images			     netG_90.pt


In [ ]:
!unzip Colorization_Data.zip

In [ ]:
resnet_model = 'resnet_gray_weights.pth.tar'
generator_model = 'checkpoint-trained.pth.tar'
# discriminator_model = 'netD_no_gan_train5.pt'

In [ ]:
# net_G = ColorNet('/content/resnet_gray_weights.pth.tar')  
# net_G.to(device)    
# print('Loaded model onto GPU.') 

# if os.path.exists("/content/" + generator_model):
#   checkpoint = torch.load("/content/" + generator_model)
#   net_G.load_state_dict(checkpoint['state_dict'])
#   print("Pretrained Model loaded")

Pretrained ResNet-gray weights loaded
Loaded colorization net.
Loaded model onto GPU.
Pretrained Model loaded


In [ ]:
from dataset import *
from network import *
from loss import *

In [ ]:
size_transform = transforms.Compose([
            transforms.Resize((300, 300), Image.BICUBIC),
            transforms.CenterCrop(256),
        ])
transform = transforms.Compose([
            transforms.ToTensor()
        ])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [ ]:
train_dataset = ImageDataset(r"/content/Images/", r"/content/colorization_annotation.csv", size_transform, transform)
train_loader = DataLoader(dataset = train_dataset, batch_size = 50, num_workers = 0, shuffle = True, pin_memory = True, drop_last = False)

In [ ]:
val_dataset = ImageDataset(r"/content/Images/", r"/content/colorization_annotation.csv", size_transform, transform)
val_loader = DataLoader(dataset = val_dataset, batch_size = 32, num_workers = 0, shuffle = False, pin_memory = True, drop_last = False)

In [ ]:
print("Number of batches : ", len(train_loader), len(val_loader))
print("Number of datapoints : ", len(train_loader.dataset), len(val_loader.dataset))

Number of batches :  166 259
Number of datapoints :  8257 8257


In [ ]:
generator_model = 'netG_120.pt'
discriminator_model = 'netD_120.pt'

In [ ]:
net_G = ColorNet('None')  
net_G.to(device)    
print('Loaded model onto GPU.') 

if os.path.exists("/content/" + generator_model):
  checkpoint = torch.load("/content/" + generator_model)
  net_G.load_state_dict(checkpoint)
  print("Pretrained Model loaded")

Loaded colorization net.
Loaded model onto GPU.
Pretrained Model loaded


In [ ]:
net_D = Discriminator(in_channels = 3) 
net_D.to(device)    
print('Loaded model onto GPU.') 

if os.path.exists("/content/" + discriminator_model):
  checkpoint = torch.load("/content/" + discriminator_model)
  net_D.load_state_dict(checkpoint)
  print("Pretrained Model loaded")

Loaded model onto GPU.
Pretrained Model loaded


In [ ]:
criterion_L1 = nn.L1Loss()
criterion_BCE = nn.BCEWithLogitsLoss()
criterion_MSE = nn.MSELoss()
criterion_Perceptual = PerceptualLoss(device)

In [ ]:
opt_G = optim.Adam(net_G.parameters(), lr = 3e-4, betas = (0.5, 0.999))
opt_D = optim.Adam(net_D.parameters(), lr = 1e-5, betas = (0.5, 0.999))

In [ ]:
g_scaler = torch.cuda.amp.GradScaler()
d_scaler = torch.cuda.amp.GradScaler()

<h1>Training

In [ ]:
net_G.train()
net_D.train()

for e in range(0, 50):
  gan_loss=0.0

  g_loss_average = 0.0
  d_loss_average = 0.0
  for (gray, ab) in tqdm(train_loader):

          # Use GPU if available
    L = gray.to(device = device, dtype = torch.float32)
    ab = ab.to(device = device, dtype = torch.float32)
    out_ab = net_G(L) 

    fake = torch.cat([L, out_ab], dim = 1)
    color = torch.cat([L, ab], dim = 1)

    

    # Train Discriminator
    D_real = net_D(L, ab)
    D_fake = net_D(L, out_ab.detach()) 
    with torch.cuda.amp.autocast():
      D_real_loss = criterion_BCE(D_real, torch.ones_like(D_real))
      D_fake_loss = criterion_BCE(D_fake, torch.zeros_like(D_fake))
      D_loss = (D_real_loss + D_fake_loss) / 2

    d_loss_average += D_loss
    opt_D.zero_grad()
    d_scaler.scale(D_loss).backward()
    if e % 4 == 0:
      # print("Training discriminator")
      d_scaler.step(opt_D)
      d_scaler.update()
    
    # Train generator
    with torch.cuda.amp.autocast():
      D_fake = net_D(L, out_ab) 

      G_bce_loss = criterion_BCE(D_fake, torch.ones_like(D_fake))
      gan_loss+=G_bce_loss
      G_l1_loss = criterion_L1(out_ab, ab)
      G_perceptual_loss = criterion_Perceptual(fake, color)
      G_mse_loss = criterion_MSE(out_ab, ab)

      G_loss = ((G_bce_loss*1) + (G_perceptual_loss*1) + (G_mse_loss*1) +  (G_l1_loss*1))

    g_loss_average += G_loss
    opt_G.zero_grad()
    g_scaler.scale(G_loss).backward()
    g_scaler.step(opt_G)
    g_scaler.update()

  print(f"Epoch {e}:")
  print(f"Generator Loss: {g_loss_average/len(train_loader):.5f}")
  print(f"Discriminator Loss: {d_loss_average/len(train_loader):.5f}")
  print(f"Generator GAN Loss: {gan_loss/len(train_loader):.5f}")

  g_path = "netG_" + str(e) + ".pt"
  d_path = "netD_" + str(e) + ".pt"
  torch.save(net_G.state_dict(), "/content/" + g_path)
  torch.save(net_D.state_dict(), "/content/" + d_path)
  print("------------------------------------------------------------------------------------------------------")
save_model(g_path)
save_model(d_path)

  0%|          | 0/166 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
 29%|██▉       | 48/166 [03:18<07:49,  3.98s/it]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
100%|██████████| 166/166 [11:05<00:00,  4.01s/it]


Epoch 0:
Generator Loss: 1.11710
Discriminator Loss: 0.69132
Generator GAN Loss: 0.70195
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [11:30<00:00,  4.16s/it]


Epoch 1:
Generator Loss: 1.11438
Discriminator Loss: 0.69759
Generator GAN Loss: 0.69000
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [10:59<00:00,  3.98s/it]


...

100%|██████████| 166/166 [11:10<00:00,  4.04s/it]


Epoch 48:
Generator Loss: 1.12152
Discriminator Loss: 0.68589
Generator GAN Loss: 0.71028
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [11:11<00:00,  4.05s/it]


Epoch 49:
Generator Loss: 1.10591
Discriminator Loss: 0.71895
Generator GAN Loss: 0.65943
------------------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored

In [ ]:
net_G.train()
net_D.train()

for e in range(50, 91):
  gan_loss=0.0

  g_loss_average = 0.0
  d_loss_average = 0.0
  for (gray, ab) in tqdm(train_loader):

          # Use GPU if available
    L = gray.to(device = device, dtype = torch.float32)
    ab = ab.to(device = device, dtype = torch.float32)
    out_ab = net_G(L) 

    fake = torch.cat([L, out_ab], dim = 1)
    color = torch.cat([L, ab], dim = 1)

    

    # Train Discriminator
    D_real = net_D(L, ab)
    D_fake = net_D(L, out_ab.detach()) 
    with torch.cuda.amp.autocast():
      D_real_loss = criterion_BCE(D_real, torch.ones_like(D_real))
      D_fake_loss = criterion_BCE(D_fake, torch.zeros_like(D_fake))
      D_loss = (D_real_loss + D_fake_loss) / 2

    d_loss_average += D_loss
    opt_D.zero_grad()
    d_scaler.scale(D_loss).backward()
    if e % 2 == 0:
      # print("Training discriminator")
      d_scaler.step(opt_D)
      d_scaler.update()
    
    # Train generator
    with torch.cuda.amp.autocast():
      D_fake = net_D(L, out_ab) 

      G_bce_loss = criterion_BCE(D_fake, torch.ones_like(D_fake))
      gan_loss+=G_bce_loss
      G_l1_loss = criterion_L1(out_ab, ab)
      G_perceptual_loss = criterion_Perceptual(fake, color)
      G_mse_loss = criterion_MSE(out_ab, ab)

      G_loss = ((G_bce_loss*1) + (G_perceptual_loss*1) + (G_mse_loss*1) + + (G_l1_loss*1))

    g_loss_average += G_loss
    opt_G.zero_grad()
    g_scaler.scale(G_loss).backward()
    g_scaler.step(opt_G)
    g_scaler.update()

  print(f"Epoch {e}:")
  print(f"Generator Loss: {g_loss_average/len(train_loader):.5f}")
  print(f"Discriminator Loss: {d_loss_average/len(train_loader):.5f}")
  print(f"Generator GAN Loss: {gan_loss/len(train_loader):.5f}")

  g_path = "netG_" + str(e) + ".pt"
  d_path = "netD_" + str(e) + ".pt"
  torch.save(net_G.state_dict(), "/content/" + g_path)
  torch.save(net_D.state_dict(), "/content/" + d_path)
  print("------------------------------------------------------------------------------------------------------")


  0%|          | 0/166 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
  2%|▏         | 4/166 [00:14<09:35,  3.56s/it]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
100%|██████████| 166/166 [09:46<00:00,  3.53s/it]


Epoch 50:
Generator Loss: 1.13566
Discriminator Loss: 0.68310
Generator GAN Loss: 0.71314
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [09:46<00:00,  3.53s/it]


Epoch 51:
Generator Loss: 1.10063
Discriminator Loss: 0.69656
Generator GAN Loss: 0.67259
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [09:40<00:00,  3.50s/it]


...

Epoch 88:
Generator Loss: 1.16461
Discriminator Loss: 0.68166
Generator GAN Loss: 0.72557
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [09:40<00:00,  3.50s/it]


Epoch 89:
Generator Loss: 1.08852
Discriminator Loss: 0.72291
Generator GAN Loss: 0.62097
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [09:30<00:00,  3.44s/it]

Epoch 90:
Generator Loss: 1.16514
Discriminator Loss: 0.67699
Generator GAN Loss: 0.72964
------------------------------------------------------------------------------------------------------


In [ ]:
net_G.train()
net_D.train()

for e in range(91, 121):
  gan_loss=0.0

  g_loss_average = 0.0
  d_loss_average = 0.0
  for (gray, ab) in tqdm(train_loader):

          # Use GPU if available
    L = gray.to(device = device, dtype = torch.float32)
    ab = ab.to(device = device, dtype = torch.float32)
    out_ab = net_G(L) 

    fake = torch.cat([L, out_ab], dim = 1)
    color = torch.cat([L, ab], dim = 1)

    

    # Train Discriminator
    D_real = net_D(L, ab)
    D_fake = net_D(L, out_ab.detach()) 
    with torch.cuda.amp.autocast():
      D_real_loss = criterion_BCE(D_real, torch.ones_like(D_real))
      D_fake_loss = criterion_BCE(D_fake, torch.zeros_like(D_fake))
      D_loss = (D_real_loss + D_fake_loss) / 2

    d_loss_average += D_loss
    opt_D.zero_grad()
    d_scaler.scale(D_loss).backward()
    if e % 2 == 0:
      # print("Training discriminator")
      d_scaler.step(opt_D)
      d_scaler.update()
    
    # Train generator
    with torch.cuda.amp.autocast():
      D_fake = net_D(L, out_ab) 

      G_bce_loss = criterion_BCE(D_fake, torch.ones_like(D_fake))
      gan_loss+=G_bce_loss
      G_l1_loss = criterion_L1(out_ab, ab)
      G_perceptual_loss = criterion_Perceptual(fake, color)
      G_mse_loss = criterion_MSE(out_ab, ab)

      G_loss = ((G_bce_loss*1) + (G_perceptual_loss*1) + (G_mse_loss*1) + + (G_l1_loss*1))

    g_loss_average += G_loss
    opt_G.zero_grad()
    g_scaler.scale(G_loss).backward()
    g_scaler.step(opt_G)
    g_scaler.update()

  print(f"Epoch {e}:")
  print(f"Generator Loss: {g_loss_average/len(train_loader):.5f}")
  print(f"Discriminator Loss: {d_loss_average/len(train_loader):.5f}")
  print(f"Generator GAN Loss: {gan_loss/len(train_loader):.5f}")

  g_path = "netG_" + str(e) + ".pt"
  d_path = "netD_" + str(e) + ".pt"
  torch.save(net_G.state_dict(), "/content/" + g_path)
  torch.save(net_D.state_dict(), "/content/" + d_path)
  print("------------------------------------------------------------------------------------------------------")


  0%|          | 0/166 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
 20%|█▉        | 33/166 [01:53<07:45,  3.50s/it]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
100%|██████████| 166/166 [09:41<00:00,  3.51s/it]


Epoch 91:
Generator Loss: 1.10773
Discriminator Loss: 0.72579
Generator GAN Loss: 0.62143
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [10:12<00:00,  3.69s/it]


Epoch 92:
Generator Loss: 1.16802
Discriminator Loss: 0.67870
Generator GAN Loss: 0.72583
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [10:11<00:00,  3.68s/it]


...

Epoch 118:
Generator Loss: 1.41767
Discriminator Loss: 0.56370
Generator GAN Loss: 0.92430
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [10:12<00:00,  3.69s/it]


Epoch 119:
Generator Loss: 0.80594
Discriminator Loss: 0.99209
Generator GAN Loss: 0.31126
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [10:16<00:00,  3.71s/it]

Epoch 120:
Generator Loss: 1.44022
Discriminator Loss: 0.54541
Generator GAN Loss: 0.95483
------------------------------------------------------------------------------------------------------


In [ ]:
net_G.train()
net_D.train()

for e in range(121, 151):
  gan_loss=0.0

  g_loss_average = 0.0
  d_loss_average = 0.0
  for (gray, ab) in tqdm(train_loader):

          # Use GPU if available
    L = gray.to(device = device, dtype = torch.float32)
    ab = ab.to(device = device, dtype = torch.float32)
    out_ab = net_G(L) 

    fake = torch.cat([L, out_ab], dim = 1)
    color = torch.cat([L, ab], dim = 1)

    

    # Train Discriminator
    D_real = net_D(L, ab)
    D_fake = net_D(L, out_ab.detach()) 
    with torch.cuda.amp.autocast():
      D_real_loss = criterion_BCE(D_real, torch.ones_like(D_real))
      D_fake_loss = criterion_BCE(D_fake, torch.zeros_like(D_fake))
      D_loss = (D_real_loss + D_fake_loss) / 2

    d_loss_average += D_loss
    opt_D.zero_grad()
    d_scaler.scale(D_loss).backward()
    if e % 2 == 0:
      # print("Training discriminator")
      d_scaler.step(opt_D)
      d_scaler.update()
    
    # Train generator
    with torch.cuda.amp.autocast():
      D_fake = net_D(L, out_ab) 

      G_bce_loss = criterion_BCE(D_fake, torch.ones_like(D_fake))
      gan_loss+=G_bce_loss
      G_l1_loss = criterion_L1(out_ab, ab)
      G_perceptual_loss = criterion_Perceptual(fake, color)
      G_mse_loss = criterion_MSE(out_ab, ab)

      G_loss = ((G_bce_loss*1) + (G_perceptual_loss*1) + (G_mse_loss*1) + + (G_l1_loss*1))

    g_loss_average += G_loss
    opt_G.zero_grad()
    g_scaler.scale(G_loss).backward()
    g_scaler.step(opt_G)
    g_scaler.update()

  print(f"Epoch {e}:")
  print(f"Generator Loss: {g_loss_average/len(train_loader):.5f}")
  print(f"Discriminator Loss: {d_loss_average/len(train_loader):.5f}")
  print(f"Generator GAN Loss: {gan_loss/len(train_loader):.5f}")

  g_path = "netG_" + str(e) + ".pt"
  d_path = "netD_" + str(e) + ".pt"
  torch.save(net_G.state_dict(), "/content/" + g_path)
  torch.save(net_D.state_dict(), "/content/" + d_path)
  print("------------------------------------------------------------------------------------------------------")


  0%|          | 0/166 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
 68%|██████▊   | 113/166 [06:35<03:10,  3.59s/it]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
100%|██████████| 166/166 [09:55<00:00,  3.59s/it]


Epoch 121:
Generator Loss: 1.04808
Discriminator Loss: 0.80054
Generator GAN Loss: 0.51935
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [10:40<00:00,  3.86s/it]


Epoch 122:
Generator Loss: 1.46956
Discriminator Loss: 0.55738
Generator GAN Loss: 0.94199
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [10:41<00:00,  3.86s/it]


...

Epoch 148:
Generator Loss: 2.13332
Discriminator Loss: 0.34030
Generator GAN Loss: 1.55127
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [10:37<00:00,  3.84s/it]


Epoch 149:
Generator Loss: 1.31084
Discriminator Loss: 0.62906
Generator GAN Loss: 0.67546
------------------------------------------------------------------------------------------------------


100%|██████████| 166/166 [10:38<00:00,  3.85s/it]

Epoch 150:
Generator Loss: 2.10842
Discriminator Loss: 0.32907
Generator GAN Loss: 1.54721
------------------------------------------------------------------------------------------------------


In [ ]:
save_model("netG_150.pt")
save_model("netD_150.pt")

Mounted at /content/drive
Model Saved
Mounted at /content/drive
Model Saved


In [ ]:
if __name__ == '__main__':
  print('Starting validation.')    
    # Switch model to validation mode
  net_G.eval()


    
  for (gray, ab) in tqdm(train_loader):

        # Use GPU if available
    L = gray.to(device = device, dtype = torch.float32)
    ab = ab.to(device = device, dtype = torch.float32)
    output = net_G(L) # throw away class predictions
        
    fake = torch.cat([L, output], dim = 1).detach().cpu().numpy()
    for i in range(fake.shape[0]):
      color_image = fake[i]
      color_image = color_image.transpose((1, 2, 0))
      color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
      color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128
      color_image = lab2rgb(color_image.astype(np.float64))
      plt.axis(False)
      plt.imshow(color_image)
      plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
if __name__ == '__main__':
  print('Starting validation.')    
    # Switch model to validation mode
  net_G.eval()


    
  for (gray, ab) in tqdm(train_loader):

        # Use GPU if available
    L = gray.to(device = device, dtype = torch.float32)
    ab = ab.to(device = device, dtype = torch.float32)
    output = net_G(L) # throw away class predictions
        
    fake = torch.cat([L, output], dim = 1).detach().cpu().numpy()
    for i in range(fake.shape[0]):
      color_image = fake[i]
      color_image = color_image.transpose((1, 2, 0))
      color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
      color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128
      color_image = lab2rgb(color_image.astype(np.float64))
      plt.axis(False)
      plt.imshow(color_image)
      plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!gdown --id 1L_lb8960cR494d3ENU9thoJAB0nLB8cR #kota

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1L_lb8960cR494d3ENU9thoJAB0nLB8cR
To: /content/Kota.zip
100% 615M/615M [00:05<00:00, 123MB/s]


In [ ]:
!unzip Kota.zip

In [ ]:
kota = ImageDataset(r"/content/Kota", r"/content/Kota/Annotation File/kota.csv", size_transform, transform)
kota_loader = DataLoader(dataset = kota, batch_size = 32, num_workers = 0, shuffle = False, pin_memory = True, drop_last = False)

In [ ]:
net_G = ColorNet('None')  
net_G.to(device)    
print('Loaded model onto GPU.') 

if os.path.exists("/content/" + "netG_99.pt"):
  checkpoint = torch.load("/content/" + "netG_99.pt")
  net_G.load_state_dict(checkpoint)
  print("Pretrained Model loaded")

Loaded colorization net.
Loaded model onto GPU.
Pretrained Model loaded


In [ ]:
if __name__ == '__main__':
  print('Starting validation.')    
    # Switch model to validation mode
  net_G.eval()


    
  for (gray, ab) in tqdm(kota_loader):

        # Use GPU if available
    L = gray.to(device = device, dtype = torch.float32)
    ab = ab.to(device = device, dtype = torch.float32)
    output = net_G(L) # throw away class predictions
        
    fake = torch.cat([L, output], dim = 1).detach().cpu().numpy()
    for i in range(fake.shape[0]):
      color_image = fake[i]
      color_image = color_image.transpose((1, 2, 0))
      color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
      color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128
      color_image = lab2rgb(color_image.astype(np.float64))
      plt.axis(False)
      plt.imshow(color_image)
      plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!gdown --id 1vngcnqJEDXevkNljmE2YsXSUTWYQOoy1
!unzip madhubala_frames.zip

In [ ]:
size_transform = transforms.Compose([
        ])
transform = transforms.Compose([
            transforms.ToTensor()
        ])

In [ ]:
frames = ImageDataset(r"/content/madhubala_frames", r"/content/test.csv", size_transform, transform)
frames_loader = DataLoader(dataset = frames, batch_size = 2, num_workers = 0, shuffle = False, pin_memory = True, drop_last = False)

In [ ]:
print(len(frames_loader), len(frames_loader.dataset))

3337 6674


In [ ]:
if __name__ == '__main__':
  print('Starting validation.')    
    # Switch model to validation mode
  net_G.eval()


    
  for (gray, ab) in tqdm(frames_loader):

        # Use GPU if available
    L = gray.to(device = device, dtype = torch.float32)
    ab = ab.to(device = device, dtype = torch.float32)
    output = net_G(L) # throw away class predictions
        
    fake = torch.cat([L, output], dim = 1).detach().cpu().numpy()
    for i in range(fake.shape[0]):
      color_image = fake[i]
      color_image = color_image.transpose((1, 2, 0))
      color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
      color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128
      color_image = lab2rgb(color_image.astype(np.float64))

      plt.axis(False)
      plt.imshow(color_image)
      plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!zip -r GAN_models50_90.zip *.pt 

In [ ]:
save_model("GAN_models50_90.zip")

Mounted at /content/drive
Model Saved


In [ ]:
!ls -lrth *.zip

-rw-r--r-- 1 root root 1.5G Apr 16 18:48 Colorization_Data.zip
-rw-r--r-- 1 root root 5.1G Apr 17 04:43 GAN_models.zip


In [ ]:
save_model("netG_80.pt")
save_model("netD_80.pt")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model Saved
Mounted at /content/drive
Model Saved


Mounted at /content/drive
Model Saved


In [ ]:
# !rm -rf netD_*

In [ ]:
!ls

checkpoint-trained.pth.tar   Images    loss.py	    resnet_gray_weights.pth.tar
colorization_annotation.csv  Kota      network.py   sample_data
dataset.py		     Kota.zip  __pycache__
